In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import precision_score, recall_score,accuracy_score,f1_score,roc_auc_score,mean_squared_error
from sklearn.metrics import log_loss
data=pd.read_csv(r'D:\书籍资料整理\kaggle\titanic\train.csv')
data=data[['PassengerId','Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']] #去掉可能的无关值

In [2]:
data['Age']=data['Age'].fillna(0)

In [3]:

sex=LabelEncoder()
sex.fit(data['Sex'])
data['Sex']=sex.transform(data['Sex'])

embarked=LabelEncoder()
embarked.fit(data['Embarked'])
data['Embarked']=embarked.transform(data['Embarked'])
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,2
1,2,1,1,0,38.0,1,0,71.2833,0
2,3,1,3,0,26.0,0,0,7.9250,2
3,4,1,1,0,35.0,1,0,53.1000,2
4,5,0,3,1,35.0,0,0,8.0500,2


In [4]:
from sklearn.feature_selection import SelectKBest,VarianceThreshold,chi2,f_classif,f_regression,mutual_info_classif
#1.方差选择

X=data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
var_filter=VarianceThreshold()
var_filter.fit_transform(X)
#可以看到对于 分类变量方差本来就是比较小的,然后对方差较小的value count偏移的不是十分厉害
var_filter.variances_

array([6.98230591e-01, 2.28218083e-01, 8.00000000e+01, 1.21467827e+00,
       6.48999031e-01, 5.12329200e+02, 6.30094435e-01])

In [5]:
print(data['Pclass'].value_counts())
print(data['Sex'].value_counts())
print(data['Parch'].value_counts())
print(data['Embarked'].value_counts())

3    491
1    216
2    184
Name: Pclass, dtype: int64
1    577
0    314
Name: Sex, dtype: int64
0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64
2    644
0    168
1     77
3      2
Name: Embarked, dtype: int64


In [6]:
#2.卡方检验  自变量离散变量
X_cat=data[['Pclass','Sex','SibSp','Parch','Embarked']] 
print(X_cat.head())
y=data['Survived']

ka_f=chi2(X_cat,y)

print(ka_f)
s_k=SelectKBest(chi2, k=4)

   Pclass  Sex  SibSp  Parch  Embarked
0       3    1      1      0         2
1       1    0      1      0         0
2       3    0      0      0         2
3       1    0      1      0         2
4       3    1      0      0         2
(array([30.87369944, 92.70244698,  2.58186538, 10.09749911,  9.75545583]), array([2.75378563e-08, 6.07783826e-22, 1.08094210e-01, 1.48470676e-03,
       1.78791305e-03]))


In [7]:
#3.ANOVA 自变量离散变量
X_col=data[['Age','Fare']] 
print(X_col.head())
y=data['Survived']

f_c=f_classif(X_col,y)

print(f_c)
s_k=SelectKBest(f_classif, k=2)

#SibSp,Age 并不是很显著.

    Age     Fare
0  22.0   7.2500
1  38.0  71.2833
2  26.0   7.9250
3  35.0  53.1000
4  35.0   8.0500
(array([ 0.09875671, 63.03076423]), array([7.53400497e-01, 6.12018934e-15]))


In [8]:
#4.试下互信息
X=data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
print(X.head())
y=data['Survived']

mic=mutual_info_classif(X,y)

print(mic)
#根据互信息 应该去掉Age，Parch

   Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0       3    1  22.0      1      0   7.2500         2
1       1    0  38.0      1      0  71.2833         0
2       3    0  26.0      0      0   7.9250         2
3       1    0  35.0      1      0  53.1000         2
4       3    1  35.0      0      0   8.0500         2
[0.047647   0.16718326 0.02701997 0.00945498 0.00838303 0.1217667
 0.01155197]


In [9]:
data_1=data[['PassengerId','Survived','Pclass','Sex','Parch','Fare','Embarked']] #去掉可能的无关值
data_1.head()

,PassengerId,Survived,Pclass,Sex,Parch,Fare,Embarked
0,1,0,3,1,0,7.2500,2
1,2,1,1,0,0,71.2833,0
2,3,1,3,0,0,7.9250,2
3,4,1,1,0,0,53.1000,2
4,5,0,3,1,0,8.0500,2


In [10]:
X=data_1[[x for x in data_1.columns if x not in ['Survived','PassengerId']]]
y=data_1['Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=49)
xgb_reg = xgb.XGBClassifier()
xgb_param = xgb_reg.get_xgb_params()
xgtrain = xgb.DMatrix(X_train, label=y_train)
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=10, nfold=5,
             metrics={'error'}, seed=0,
             callbacks=[xgb.callback.print_evaluation(show_stdv=False)
                       # ,xgb.callback.early_stop(3)
                       ])
xgb_reg.set_params(n_estimators=cvresult.shape[0])#把clf的参数设置成最好的树对应的参数
xgb_reg.fit(X_train, y_train, eval_metric='auc')
y_pred = xgb_reg.predict(X_val)
y_score = xgb_reg.predict_proba(X_val)

[0]	train-error:0.15719	test-error:0.20062
[1]	train-error:0.15270	test-error:0.20061
[2]	train-error:0.15195	test-error:0.20215
[3]	train-error:0.15232	test-error:0.20962
[4]	train-error:0.15158	test-error:0.20962
[5]	train-error:0.15120	test-error:0.20961
[6]	train-error:0.14896	test-error:0.20811
[7]	train-error:0.14783	test-error:0.20212
[8]	train-error:0.14558	test-error:0.20062
[9]	train-error:0.14484	test-error:0.20958


C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)
C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [11]:
print('测试集准确率:',accuracy_score(y_val, y_pred))
print('测试集精度:',precision_score(y_val, y_pred))
print('测试集召回率:',recall_score(y_val, y_pred))
print('auc:',roc_auc_score(y_val, y_score[:,1]))

测试集准确率: 0.8609865470852018
测试集精度: 0.8260869565217391
测试集召回率: 0.75
auc: 0.9031059792337988


In [12]:
# 互信息准则做xgboost
data_2=data[['PassengerId','Survived','Pclass','Sex','SibSp','Fare','Embarked']] #去掉可能的无关值

X=data_2[[x for x in data_2.columns if x not in ['Survived','PassengerId']]]
y=data_2['Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=49)
xgb_reg2 = xgb.XGBClassifier()
xgb_param = xgb_reg2.get_xgb_params()
xgtrain = xgb.DMatrix(X_train, label=y_train)
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=10, nfold=5,
             metrics={'error'}, seed=0,
             callbacks=[xgb.callback.print_evaluation(show_stdv=False)
                       # ,xgb.callback.early_stop(3)
                       ])
xgb_reg2.set_params(n_estimators=cvresult.shape[0])#把clf的参数设置成最好的树对应的参数
xgb_reg2.fit(X_train, y_train, eval_metric='auc')
y_pred = xgb_reg2.predict(X_val)
y_score = xgb_reg2.predict_proba(X_val)
print('测试集准确率:',accuracy_score(y_val, y_pred))
print('测试集精度:',precision_score(y_val, y_pred))
print('测试集召回率:',recall_score(y_val, y_pred))
print('auc:',roc_auc_score(y_val, y_score[:,1]))

[0]	train-error:0.16243	test-error:0.19617
[1]	train-error:0.15270	test-error:0.20065
[2]	train-error:0.15382	test-error:0.20513


C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


[3]	train-error:0.15270	test-error:0.20362
[4]	train-error:0.14858	test-error:0.20214
[5]	train-error:0.14895	test-error:0.20213
[6]	train-error:0.14746	test-error:0.20513
[7]	train-error:0.14821	test-error:0.20361
[8]	train-error:0.14371	test-error:0.20512
[9]	train-error:0.14596	test-error:0.20810
测试集准确率: 0.8475336322869955
测试集精度: 0.8088235294117647
测试集召回率: 0.7236842105263158
auc: 0.8896348012889367


C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [13]:
#生成
data_test=pd.read_csv(r'D:\书籍资料整理\kaggle\titanic\test.csv')
data_test=data_test[['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']] #去掉可能的无关值
data_test['Sex']=sex.transform(data_test['Sex'])
data_test['Embarked']=embarked.transform(data_test['Embarked'])
data_test.head()
X_test=data_test[[x for x in data_test.columns if x not in ['PassengerId','Age','SibSp']]]
y_test_pre=xgb_reg.predict(X_test)
data_test['Survived']=y_test_pre
data_test.head()
data_test=data_test[['PassengerId','Survived']]
data_test.to_csv(r'D:\书籍资料整理\kaggle\titanic\output.csv',index=False)

In [14]:
#生成
data_test=pd.read_csv(r'D:\书籍资料整理\kaggle\titanic\test.csv')
data_test=data_test[['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']] #去掉可能的无关值
data_test['Sex']=sex.transform(data_test['Sex'])
data_test['Embarked']=embarked.transform(data_test['Embarked'])
data_test.head()
X_test=data_test[[x for x in data_test.columns if x not in ['PassengerId','Age','Parch']]]
y_test_pre=xgb_reg2.predict(X_test)
data_test['Survived']=y_test_pre
data_test.head()
data_test=data_test[['PassengerId','Survived']]
data_test.to_csv(r'D:\书籍资料整理\kaggle\titanic\output1.csv',index=False)